<a href="https://colab.research.google.com/github/TLEtoAC/Prompt-Classification/blob/main/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install Ollama (the application)
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Install all Python libraries
!pip install ollama flask flask-cors pyngrok


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
# 1. Start the Ollama service
# This command forces the server to run in the background
# and throws away its log, so the cell doesn't get "stuck".
!ollama serve > /dev/null 2>&1 &

# 2. Wait 3 seconds to be sure it's booted
!sleep 3

print("✅ Ollama service is running in the background.")

✅ Ollama service is running in the background.


In [17]:
# 2. Pull the model (this will be fast if it's already downloaded)
#!ollama pull phi3:mini
!ollama pull llama3:8b
!ollama pull gemma2:2b

In [18]:
!ollama run llama3:8b  "What is 1 + 1?"

The answer to 1 + 1 is... 2!



In [ ]:
import ollama
import sys
import re
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import os

# ===================================================================
# 1. NGROK AUTHENTICATION
# ===================================================================
# GET_NGROK_TOKEN
# 1. Go to https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Copy your token and paste it here:
NGROK_AUTHTOKEN = "34smbDvZUnlyZFGvBe3ABlv5yMN_38Zc8zgxQpnfQN1ezyfVE" # 🚨 PASTE YOUR TOKEN

if "YOUR_NGROK_AUTHTOKEN_HERE" in NGROK_AUTHTOKEN:
    print("\n" + "="*50)
    print("❌ ERROR: Please get your ngrok authtoken.")
    print("Go to https://dashboard.ngrok.com/get-started/your-authtoken")
    print("Then paste it into the NGROK_AUTHTOKEN variable and re-run this cell.")
    print("="*50)
    sys.exit()
else:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)

# ===================================================================
# 2. PHI-3 CLASSIFIER FUNCTION (FIXED PROMPT)
# ===================================================================

# FIX: We are putting all the examples *inside* the system prompt.
# This is much more reliable than sending them as chat history.
NEW_SYSTEM_PROMPT = """
You are a prompt classification router. Your only job is to analyze the user's
prompt and respond with a *single word*: low, medium, or high.
Do not explain your choice. Do not add any other text.

Here are your rules and examples:

---
EXAMPLES
---
User: "What is 1 + 1?"
Assistant: low
---
User: "who was the president in 1999?"
Assistant: low
---
User: "Why is microservices significant?"
Assistant: medium
---
User: "Explain quantum entanglement in simple terms."
Assistant: medium
---
User: "List the top 5 programming languages for machine learning."
Assistant: medium
---
User: "Summarize the following research paper abstract..."
Assistant: high
---
User: "Write a short story about a sad robot."
Assistant: high
---
User: "Compare the ethical implications of AI in healthcare vs autonomous vehicles."
Assistant: high
---
"""

def predict_complexity(prompt_to_classify: str) -> str:

    # We now send *only* the new system prompt and the single user prompt.
    messages = [
        {'role': 'system', 'content': NEW_SYSTEM_PROMPT},
        {'role': 'user', 'content': prompt_to_classify}
    ]

    try:
        # --- UPGRADED MODEL ---
        # We are using a more powerful "teacher" model for better data quality
        response = ollama.chat(
            model='llama3:8b', # Switched from phi3:mini to llama3:8b
            messages=messages,
            options={'num_ctx': 2048, 'temperature': 0.0} # Increased context
        )

        classification = response['message']['content'].lower().strip()
        words = re.split(r'[^a-z]+', classification)
        first_word = words[0] if words else ""

        if first_word in ["low", "medium", "high"]:
            return first_word

        print(f"Warning: Model gave unexpected output: '{classification}'", file=sys.stderr)
        return "medium" # Default

    except Exception as e:
        print(f"❌ Error during classification: {e}", file=sys.stderr)
        return "error"

# ===================================================================
# 3. CREATE THE FLASK API SERVER
# ===================================================================
app = Flask(__name__)
CORS(app)

@app.route('/classify', methods=['POST'])
def handle_classification():
    print("Backend: Received a request...")
    try:
        data = request.json
        prompt = data.get('prompt')

        if not prompt:
            print("Backend: Error - No prompt provided")
            return jsonify({"error": "No prompt provided"}), 400

        # --- This is where the magic happens ---
        complexity = predict_complexity(prompt)
        # --- ---

        if complexity == "error":
            print("Backend: Error - Classification failed.")
            return jsonify({"error": "Classifier failed to connect to Ollama"}), 500

        # --- UPDATED MODEL MAP ---
        model_map = {
            "low": "gemma2:2b",    # Smallest model for simple tasks
            "medium": "phi3:mini",   # Use phi3 for medium tasks
            "high": "gpt-4o-api"   # "Expensive" API model
        }
        selected_model = model_map.get(complexity, "phi3:mini") # Default to phi3:mini

        print(f"Backend: Classified as '{complexity}', selected model '{selected_model}'")

        return jsonify({
            "complexity": complexity,
            "selected_model": selected_model
        })
    except Exception as e:
        print(f"Backend: Server Error: {e}", file=sys.stderr)
        return jsonify({"error": str(e)}), 500

# ===================================================================
# 4. START NGROK & RUN THE SERVER
# ===================================================================
print("\nStarting ngrok tunnel...")
public_url = ngrok.connect(5000)
print("="*50)
print("✅ YOUR BACKEND IS LIVE!")
print(f"Copy this public URL: {public_url}")
print("="*50)

app.run(port=5000)




Starting ngrok tunnel...
✅ YOUR BACKEND IS LIVE!
Copy this public URL: NgrokTunnel: "https://unmiasmic-ontogenetic-kandis.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Nov/2025 16:35:25] "OPTIONS /classify HTTP/1.1" 200 -


Backend: Received a request...


```
import requests
from bs4 import beautifulsoup

# url of the website you want to scrape
url = "https://www.example.com"

# send a get request to the website
response = requests.get(url)

# check if the response was successful (200 ok)
if response.status_code == 200:
    # parse the html content using beautifulsoup
    soup = beautifulsoup(response.content, 'html.parser')

    # find all the <h2> tags on the page
    headings = soup.find_all('h2')

    # print out the text of each heading
    for heading in headings:
        print(heading.text)

else:
    print("failed to retrieve webpage")
```
this script sends a get request to the specified url, and then uses beautifulsoup to parse the html content. it then finds all the `<h2>` tags on the page and prints out their text.

you can modify this script to scrape specific elements or data from the website by using beautifulsoup's various methods, such as:

* `find()`: find a single element with a specific tag name
* `find_all()`: find m

Backend: Classified as 'medium', selected model 'phi3:mini'


...

Backend: Classified as 'medium', selected model 'phi3:mini'


INFO:werkzeug:127.0.0.1 - - [01/Nov/2025 16:37:14] "OPTIONS /classify HTTP/1.1" 200 -


Backend: Received a request...


INFO:werkzeug:127.0.0.1 - - [01/Nov/2025 16:37:15] "POST /classify HTTP/1.1" 200 -


Backend: Classified as 'medium', selected model 'phi3:mini'
